# IMPORTS

In [ ]:
%matplotlib inline
import mpld3
mpld3.enable_notebook()

In [ ]:
from AxonDeepSeg.visualization.visualize import visualize_training
import json
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.misc import imread
#from prettytable import PrettyTable
#from ipywidgets.widgets import *

# 1/ Build the dataset for training.

In [ ]:
# input data to build the training set
dataset_name = 'SEM_3classes_reduced'

In [ ]:
path_data = '../data/' + dataset_name + '/raw/'
path_testing = '../data/humanseg/im1/1/'

# output path of training data path
path_training = '../data/' + dataset_name + '/training/'

# 2/ Load the config file and train the network

Choose here the name of the model you want to load

In [ ]:
model_name = 'baseline_fake'

In [ ]:
# optional input path of a model to initialize the training
#path_model_init = 'network_testing/test_2905'
path_model_init = None

# output path for trained U-Net
path_model = '../models/' + model_name

path_configfile = path_model + '/config_network.json'

if not os.path.exists(path_model):
    os.makedirs(path_model)

with open(path_configfile, 'r') as fd:
    config_network = json.loads(fd.read())
    
# OPTIONAL : specify the gpu one wants to use.
gpu_device = 'gpu:0' # or gpu_device = 'gpu:1' these are the only two possible targets for now.

In [ ]:
n_classes = config_network['network_n_classes']

# TRAINING OF THE NETWORK

Uniquement nécessaire si le réseau n'a pas encore été entrainé. Sinon on va juste utiliser le modèle existant dans models

from AxonDeepSeg.train_network import train_model
train_model(path_training, path_model, config_network, path_model_init=None, gpu = gpu_device)

# 3/ Apply the model to segment one image

#### Segmentation

In [ ]:
from AxonDeepSeg.apply_model import axon_segmentation
_, pred_proba = axon_segmentation(path_testing, path_model, config_network, pred_proba=True)

# TRANSFORMING PRED_PROBA INTO REAL PROBABILITIES

In [ ]:
a = np.exp(pred_proba)
b = np.sum(a, axis=-1)
c = np.stack([np.divide(a[:,:,i],b) for i in range(3)], axis=-1)

# VISUALISATION OF THE SEGMENTATION

Import images

In [ ]:
imorg = plt.imread(path_testing + '/image.png')
imads = plt.imread(path_testing + '/AxonDeepSeg.png')
mask = imread(path_testing + '/mask.png', mode='L')

In [ ]:
plt.figure(figsize=(18,10))
plt.title('Original image')
plt.imshow(imorg, cmap='gray')

In [ ]:
plt.figure(figsize=(18,10))
plt.title('Segmented image')
plt.imshow(imads)

In [ ]:
plt.figure(figsize=(18,10))
plt.title('Superposed images')
plt.imshow(imorg, cmap='gray', alpha=0.5)
plt.imshow(imads, cmap='viridis', alpha=0.5)

# Visualisation of the probability maps

In [ ]:
plt.figure(figsize=(18,10))
plt.clf()
for i in range(n_classes):
    plt.subplot(2,2,i+1)
    plt.title('class ' + str(i))
    plt.imshow(c[:,:,i])
    plt.colorbar()
plt.show();

Looking more closely at the myelin predictions

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(imorg, cmap='gray', alpha=1)
plt.imshow(c[:,:,1], cmap='Reds', alpha=0.2)
plt.title('Predictions myelin')
plt.show();

In [ ]:
plt.close('all')

In [ ]:
# Computing the difference between mask and myelin
myelin = c[:,:,1]
mask_ = (mask == 128).astype(np.int)
d = mask_ - myelin

# Displaying the results
plt.figure(figsize=(13,8))
plt.imshow(d, cmap='jet')
plt.colorbar()
plt.show();

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(imads)
plt.show();